In [ ]:
from sqlalchemy import text

#FUNCTION for getting the quantity of an item
def item_status(idr): #https://stackoverflow.com/questions/13205157/just-one-row-from-sqlalchemy
    with engine.connect() as conn:
        sql = 'select * from items where id ='+str(idr)
        sql = text(sql)
        result = conn.execute(sql)
        result = result.fetchone()
        return result[3]

#FUNCTION for adding new item
def add_item():
    with engine.connect() as conn:
        name = input('Enter Item Name :')
        qty = input('Enter Quantity : ')
        price = input('Enter Price :')
        today = date.today()
        reorder = input('Enter Reorder Level :')
        sql = 'INSERT INTO items(name,price,qty,reorder) VALUES ("'+name+'",'+price+','+qty+','+reorder+' );'
        #sql2 = 'insert into transaction(dot,qty,type) values ("'+str(today)+'",'+qty+',"purchase");'
        #print(sql)
        sql = text(sql)
        result = conn.execute(sql)
        #cursor.execute(sql2)
        conn.close()
        print('\n\nNew Item ('+name+') Added Successfully')
        wait = input('\n\n\n Press any key to continue....')

#FUNCTION for subtracting from stock
def sell_item():
    with engine.connect() as conn:
        today = date.today()
        item_no = input('Enter Item ID :')
        qty = input('Enter Sold Quantity : ')
        
        '''
        item_sql = 'SELECT name FROM items WHERE id=' +item_no+ ';'
        item = conn.execute(text(item_sql))
        item = item.fetchall()[0]
        '''
        
        #UPDATE item quantity (Subtract quantity with quantity sold)
        sql = 'UPDATE items SET qty = qty - '+ qty +' WHERE id='+item_no+';'  
        
        #ADD transaction record to transaction table
        sql2 = 'insert into transaction(dot,qty,type,item_id) values ("' + \
            str(today)+'",'+qty+',"sold" , '+item_no+');'
        
        result = item_status(item_no)

        if result >= int(qty):
          conn.execute(text(sql))
          conn.execute(text(sql2))
          print('\n\nItem (['+item_no+']) Updated Successfully')
        else:
          print('We do not have this much of (['+item_no+']) in our stock')

        conn.close()
    
        wait = input('\n\n\n Press any key to continue....')

def received_item():
    with engine.connect() as conn:
        today = date.today()
        item_no = input('Enter Item ID :')
        qty = input('Enter Received Quantity : ')
        sql = 'update items set qty = qty+' + qty + ' where id='+item_no+';'
        sql2 = 'insert into transaction(dot,qty,type,item_id) values ("' + \
            str(today)+'",'+qty+',"purchase",'+item_no+');'
        #print(sql)
        conn.execute(text(sql))
        conn.close()
        print('\n\nItem updated successfully')
        wait = input('\n\n\n Press any key to continue....')

def search_item():
    with engine.connect() as conn:
        print('Search Screen ')
        print('-'*80)
        name = input('Enter Item Name :')
        sql ="select * from items where name like '%"+name+"%';"
        result = conn.execute(text(sql))
        records = result.fetchall()
        t = PrettyTable(['ID', 'Item Name', 'Price', 'Quantity', 'Reorder Level'])
        for idr, name, price, qty, reorder in records:
          t.add_row([idr, name, price, qty, reorder])

        print('Search Result for :',name)
        print(t)
        conn.close()
        wait = input('\n\n\n Press any key to continue....')
        
def search_menu():
    while True:
      print(' Search Menu')
      print("\n1.  Name wise")
      print('\n2.  back to Main Menu')
      print('\n\n')
      choice = int(input('Enter your choice ...: '))

      if choice == 1:
        search_item()
      if choice == 2:
        break

def report_items():
    with engine.connect() as conn:
        name = input('Enter Item Name :')
        sql = "select * from items;"
        result = conn.execute(text(sql))
        records = result.fetchall()
        
        print(records)
        
        print('Item List')
        t = PrettyTable(['ID', 'Item Name', 'Price', 'Quantity', 'Reorder Level'])

        for idr, name, price, qty, reorder in records:
          t.add_row([idr,name,price,qty,reorder])
        # print(idr,name,fname,add,phone,email)
        print(t)
        conn.close()
        wait = input('\n\n\n Press any key to continue....')

def report_qty_greater_reorder():
    with engine.connect() as conn:
        sql = "select * from items where qty > reorder;"
        result = conn.execute(text(sql))
        records = result.fetchall()
        
        print('Item List')
        t = PrettyTable(['ID', 'Item Name', 'Price', 'Quantity', 'Reorder Level'])

        for idr, name, price, qty, reorder in records:
          t.add_row([idr, name, price, qty, reorder])
        # print(idr,name,fname,add,phone,email)
        print(t)
        conn.close()
        wait = input('\n\n\n Press any key to continue....')

def report_qty_lesser_reorder():
    with engine.connect() as conn:
        sql = "select * from items where qty < reorder;"
        result = conn.execute(text(sql))
        records = result.fetchall()
        
        print('Item List')
        t = PrettyTable(['ID', 'Item Name', 'Price', 'Quantity', 'Reorder Level'])

        for idr, name, price, qty, reorder in records:
          t.add_row([idr, name, price, qty, reorder])
        # print(idr,name,fname,add,phone,email)
        print(t)
        conn.close()
        wait = input('\n\n\n Press any key to continue....')

def report_sold_items():
    with engine.connect() as conn:
        today = date.today()
        sql = "select id, name,dot,t.qty,price from items,transaction t where id=tid and type='sold' and dot='"+str(today)+"';"
        result = conn.execute(text(sql))
        records = result.fetchall()

        t = PrettyTable(['ID', 'Item Name', 'DOT', 'Quantity', 'Price'])
        for idr, name, dot, qty, price in records:
          t.add_row([idr, name, dot,qty,price])
        # print(idr,name,fname,add,phone,email)
        
        print('Items Sold Today :', today)
        print(t)
        conn.close()
        wait = input('\n\n\n Press any key to continue....')

def report_received_item():
    with engine.connect() as conn:
        today = date.today()
        sql = "select id, name,dot,t.qty,price from items,transaction t where id=tid and type='purchase' and dot='" + \
            str(today)+"';"
        result = conn.execute(text(sql))
        records = result.fetchall()

        t = PrettyTable(['ID', 'Item Name', 'DOT', 'Quantity', 'Price'])
        for idr, name, dot, qty, price in records:
          t.add_row([idr, name, dot, qty, price])
        # print(idr,name,fname,add,phone,email)
        
        print('Items Received Today :',today)
        print(t)
        conn.close()
        wait = input('\n\n\n Press any key to continue....')
        
def report_menu():
    while True:
      print(' Report Menu')
      print("\n1.  Item List")
      print('\n2.  sold Items - Today')
      print('\n3.  Received Items- Today')
      print('\n4.  Item Qty > Reorder')
      print('\n5.  Item Qty < Reorder')
      print('\n6.  Exit to main Menu')
      print('\n\n')
      choice = int(input('Enter your choice ...: '))

      if choice == 1:
        report_items()
      if choice == 2:
        report_sold_items()
      if choice == 3:
        report_received_item()
      if choice == 4:
        report_qty_greater_reorder()
      if choice == 5:
        report_qty_lesser_reorder()
      if choice == 6:
        break

def main_menu():
    while True:
      print(' Main Menu')
      print("\n1.  Add Items")
      print('\n2.  sell Items')
      print('\n3.  Received Items')
      print('\n4.  Search Menu')
      print('\n5.  Report Menu')
      print('\n6.  Close application')
      print('\n\n')
      choice = int(input('Enter your choice ...: '))
      
      if choice == 1:
        add_item()
      if choice == 2:
        sell_item()
      if choice == 3:
        received_item()
      if choice == 4:
        search_menu()
      if choice == 5:
        report_menu()
      if choice == 6:
        break

if __name__ == "__main__":
    main_menu()

 Main Menu

1.  Add Items

2.  sell Items

3.  Received Items

4.  Search Menu

5.  Report Menu

6.  Close application



